<a href="https://colab.research.google.com/github/bibersay/Deep-Project/blob/main/%EA%B5%AD%EB%AF%BC%EC%8B%A0%EB%AC%B8%EA%B3%A0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import numpy as np
import time

result = pd.DataFrame()
for k in range(1,100):
    url = 'https://www.epeople.go.kr/nep/prpsl/opnPrpl/opnpblPrpslList.npaid?pageIndex='+str(k)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    links = soup.select('td > a')
    count = 0
    names , keys = [],[]
    for i,lin in enumerate(links):
        if i >= 10:
            break
        name, key= lin.get('onclick').split("'")[1], lin.get('onclick').split("'")[3]
        names.append(name)
        keys.append(key)    
        url = 'https://www.epeople.go.kr/nep/prpsl/opnPrpl/opnpblPrpslView.npaid?prplRqstNo='+names[i]+'&instRcptSn='+keys[i]

        
        res = requests.get(url)
        soup2 = BeautifulSoup(res.text,'lxml')
        title = soup2.find('div', class_ ='cellBig').text.strip()
        # context = soup2.find('div', class_ ='b_cont').text.strip()
        contexts = soup2.select('div.b_conItem > div.b_cont')
        data = []
        # print(contexts)
        for content in contexts:
            data.append(content.text.strip())
        data = ''.join(data)
        if len(data) != 0:
            df1 = pd.DataFrame({'title' : title,
                            'content' : [data]
            })

            result = pd.concat([result, df1])
            result.index = np.arange(len(result))
        
        if i %4 == 0:
            print(f"sleep minute. count : {str(i)}, local time : {time.strftime('%Y-%m-%d', time.localtime(time.time()))}, {time.strftime('%X', time.localtime(time.time()))}, data length : {str(len(result))} ")

print(df1)

In [ ]:
# from google.colab import files
# result.to_csv('result.csv')
# files.download('result.csv')

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import pandas as pd
result=pd.read_csv('/content/result.csv',  lineterminator='\n')

In [ ]:
print(result.shape)
df = result
df['content'][0]

In [ ]:
import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]','',str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ 가-힣 0-9]','',str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab, Okt

In [ ]:
okt = Okt()
sentence = "안녕하세요 저는 천재입니다."
temp_X = okt.morphs(sentence)
temp_X

['안녕하세요', '저', '는', '천재', '입니다', '.']

In [ ]:
okt = Okt()
df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

In [ ]:
df['token_final'] = df.title_token + df.content_token
print(df)
print(df.dtypes)    

In [ ]:
from gensim.models import Word2Vec
df_drop = df[['title_token', 'content_token']]
embedding_model = Word2Vec(df_drop['content_token'],
                           sg =1,
                           size=100,
                           window =4,
                           min_count =1,
                           workers =2
                           )
print(embedding_model)

model_result = embedding_model.wv.most_similar("음주")
print(model_result)

Word2Vec(vocab=6885, size=100, alpha=0.025)
[('아이디어', 0.9985363483428955), ('영산강', 0.9984657764434814), ('갈수록', 0.9984551668167114), ('장착', 0.9984496831893921), ('설정', 0.9984275698661804), ('방면', 0.9984175562858582), ('대형견', 0.9984142780303955), ('길이', 0.9984104037284851), ('우려', 0.9984077215194702), ('이로', 0.9983971118927002)]


In [ ]:
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('./petitions_tokens_w2v')

loaded_model = KeyedVectors.load_word2vec_format('./petitions_tokens_w2v')
model_result = loaded_model.most_similar('음주')
print(model_result)

[('아이디어', 0.9985363483428955), ('영산강', 0.9984657764434814), ('갈수록', 0.9984551668167114), ('장착', 0.9984496831893921), ('설정', 0.9984275698661804), ('방면', 0.9984175562858582), ('대형견', 0.9984142780303955), ('길이', 0.9984104037284851), ('우려', 0.9984077215194702), ('이로', 0.9983971118927002)]


In [ ]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=.8, random_state = rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('/content/train.csv', index=False, encoding = 'utf-8-sig')
val.to_csv('/content/val.csv', index=False, encoding = 'utf-8-sig')


In [ ]:
!pip install torchtext==0.8.0

In [ ]:
from torchtext.data.utils import get_tokenizer

def tokenizer(text):
    text = re.sub('[\[\]\']','',str(text))
    text = text.split(',')
    return text

TEXT = get_tokenizer(tokenizer = tokenizer)
LABEL = get_tokenizer(tokenizer = tokenizer)